In [1]:
# part 1

import os, re, sys
import numpy as np

filename = "test1.txt"
with open(filename, 'r') as f:
    s = f.read()
    rows = s.split("\n")

nx, ny = len(rows[0]), len(rows)
board = np.array([[1 if x == "#" else 0 for x in row ] for row in rows], dtype=int)

y, x, d = ny - 2, 1, 1 # start  
goal_y, goal_x = 1, nx - 2 # finish

# for each state, how much does it cost to reach it?
location_cost = np.full((ny, nx, 4), 0, dtype=int)
location_cost[y, x, :] = [1000, 0, 1000, 2000]

# recently added states that we should explore furthest
new_locs = [(y, x, d_) for d_ in range(4)]

# explore the smallest state first!
find_smallest = lambda: new_locs[np.argmin([location_cost[loc] for loc in new_locs])]

dirs = np.array([[-1, 0], [0, 1], [1, 0], [0, -1]])
orientation_cost = [0, 1000, 2000, 1000] # a 90 degr turn costs 1000

def update(loc):
    y, x, d = loc
    dy, dx = dirs[d]
    ny, nx = y + dy, x + dx
    
    cost = location_cost[y, x, d]
    new_locs.remove((y, x, d))
    if not board[ny, nx]:          
        for i in range(4):
            if location_cost[ny, nx, i] > cost + orientation_cost[d - i] + 1 or not location_cost[ny, nx, i]:
                location_cost[ny, nx, i] = cost + orientation_cost[d - i] + 1
                new_locs.append((ny, nx, i))

while True:
    if max(location_cost[goal_y, goal_x]) > 0:
        print(location_cost[goal_y, goal_x]); break
    update(find_smallest())

[11048 12048 13048 12048]


In [2]:
# part 2

final_paths = [[int(board[y, x]) for x in range(nx)] for y in range(ny)]

def reconstruct():
    
    # start at the cheapest states at the finish location
    cost = lambda x: location_cost[goal_y, goal_x, x]
    minidx = np.where(cost(range(4)) == min(map(cost, range(4))))[0]

    locations = [(goal_y, goal_x, i) for i in minidx]
    new_locs = [(goal_y, goal_x, i) for i in minidx]
    while new_locs:
        loc = new_locs.pop()
        y, x, d = loc
        possible_prev = [(y + 1, x, 0), (y, x - 1, 1), (y - 1, x, 2), (y, x + 1, 3)]

        # for each of the surrounding edges, check if it is possible we come from there
        for i in range(4):
            prev = possible_prev[i]
            if location_cost[prev] == location_cost[y, x, d] - 1 - orientation_cost[d - i] and location_cost[prev] != 0:
                new_locs.append(prev)
                locations.append(prev)

                final_paths[prev[0]][prev[1]] = 8
    
    # unique locations (each location appears up to 4 times, for each direction)
    print(len(set(loc[:2] for loc in locations))) 

reconstruct()


64


In [3]:
# visualisation of all the tiles that are part of a shortest path
for row in final_paths:
    for val in row:
        print({0: " ", 1: "0", 8: "*"}[val], end='')
    print('')

00000000000000000
0   0   0   0   0
0 0 0 0 0 0 0 0*0
0 0 0 0   0   0*0
0 0 0 0 000 0 0*0
0***0 0 0     0*0
0*0*0 0 0 00000*0
0*0*  0 0 0*****0
0*0*00000 0*000*0
0*0*0  *****0***0
0*0*000*00000*000
0*0*0***0  ***0 0
0*0*0*00000*000 0
0*0*0*******  0 0
0*0*0*000000000 0
0*0***          0
00000000000000000
